# Посещение врачей

In [1]:
import pandas as pd
import numpy as np

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")

%load_ext lab_black

ISO-8859-1

# Если нестандартные кодировки:

In [313]:
# загрузка файла для ЕАО !!!
df_data = pd.read_csv(
    "/home/sk27/work/med/data/source.csv",
    sep=",",
    encoding="utf-16",
    low_memory=False,
)

year,month,region,clinic,oid,depart,branch,doctor_specialty,doctor_profile,terms,doctor_id,diagnosis,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum

## Заголовок добавить вручную
### Переходить сразу к расшифровке кодов

In [314]:
df_data

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,1,1,1,NaN
1,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.3,R03.03.102.001,137-097-107 64,24,790001,79000100,1,1,1,NaN
2,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,1.0,B01.014.001,137-097-107 64,24,790001,79000100,10,1,1,NaN
3,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.002,137-097-107 64,24,790001,79000100,12,5,1,NaN
4,2023,1,99,99000000000,790001,1.2.643.5.1.13.13.12.2.79.8850,79000100,NaN,24,28,...,3.0,B01.014.002,138-993-968 45,24,790001,79000100,2,15,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25405,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,41,60,...,1.0,B01.027.001,17717264189,41,790021,79002100,2,3,1,NaN
25406,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,45,162,...,1.0,B01.028.001,13109077827,45,790021,79002100,1,1,1,NaN
25407,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,76,97,...,1.0,B01.047.001,6180038532,76,790021,79002100,1,1,1,NaN
25408,2023,2,99,99000000000,790021,1.2.643.5.1.13.13.12.2.79.8838,79002100,NaN,76,97,...,1.0,B01.047.001,6180038532,76,790021,79002100,1,1,1,NaN


In [315]:
# количество строк и столбцов
df_data.shape

(25410, 23)

In [316]:
# повторяющиеся строки
df_data[df_data.duplicated()]

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum


In [317]:
# количество повторяющихся строк
df_data[df_data.duplicated()].shape

(0, 23)

In [318]:
# замена истинно постых значений словом nan
# это нужно для подсчёта повторяющихся строк
df_data = df_data.fillna("nan")
# после этого переходим к подсчёту повторяющихся строк

# Обычные кодировки
## 1. Подготовка данных

In [15]:
%%time

# вспомогательные функции

def oktmo(o):
    """
    функция дополняет код ОКТМО до 11 символов или до 10 символов без ведущего 0
    """
    if len(str(o)) == 1:
        l = 9
    else:
        l = 11-len(str(o))
    return int(str(o) + "0"*(l))
    
def visit_count(row):
    """
    функция определяет количество посещений согласно условиям
    """
    if row["purpose"] >= 3:
        res = row["code_usl"] * row["kol_usl"] * row["size"]
    else:
        res = row["sl_id"] * row["size"]
    if np.isnan(res):
        return "не определено"
    else:
        return res
    
# добавление в файл данных строки заголовков
with open("/home/sk27/work/med/data/source.csv", "r") as f:
    first_line = next(f).strip()
    if first_line[0:4] != "year":
        with open(
            "/home/sk27/work/med/data/source.csv", "r", encoding="ISO-8859-1"
        ) as original:
            data = original.read()
        with open("/home/sk27/work/med/data/source.csv", "w") as modified:
            modified.write(
                "year;month;region;clinic;oid;depart;branch;doctor_specialty;doctor_profile;terms;doctor_id;diagnosis;purpose;service_id;usl_service_id;usl_doctor_specialty;usl_clinic;usl_depart;sl_id;code_usl;kol_usl;yet_sum\n"
                + data
            )
    else:
        pass

# загрузка файла
df_data = pd.read_csv(
    "/home/sk27/work/med/data/source.csv",
    sep=";",
    encoding="ISO-8859-1",
    low_memory=False,
)


# удаление кавычек
for i, col in enumerate(df_data.columns):
    df_data.iloc[:, i] = df_data.iloc[:, i].astype("str").str.replace('"', "")
# df_data = df_data.apply(pd.to_numeric, errors="ignore")

# замена ВСЕХ запятых точками
for i, col in enumerate(df_data.columns):
    df_data.iloc[:, i] = df_data.iloc[:, i].astype("str").str.replace(',', ".")
df_data = df_data.apply(pd.to_numeric, errors="ignore")

# вставка столбца ОКТМО
df_data.insert(3, "oktmo", df_data["region"].apply(oktmo))

# проверка загруженных данных

# а) проверка количества столбцов - их должно быть именно 23 вместе с добавленным мной столбцом ОКТМО
if df_data.shape[1] > 23:
    print(f"Количество полей в файле (их {df_data.shape[1]}) не соответствует форме отчёта, должно быть 23.")
else:
    pass

# б) проверка столбца terms на "только 3"
try:
    if df_data["terms"].mean() != 3:
        print(
            "Нарушено требование об указании в поле 'Условия оказания медпомощи' только значения '3'."
        )
except:
    print("Не проходит проверка на '3'")

CPU times: user 4.28 s, sys: 71 ms, total: 4.36 s
Wall time: 4.35 s


In [16]:
# контроль набора данных
df_data

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2023,1,79000,79000000000,10003,1.2.643.5.1.13.13.12.2.1.13,nan,nan,17,16,...,1.0,450,077-363-427 87,17,10003,nan,1,1,1,0.0
1,2023,1,79000,79000000000,10003,1.2.643.5.1.13.13.12.2.1.13,nan,nan,17,16,...,1.0,450,077-363-427 87,17,10003,nan,1,1,1,0.0
2,2023,1,79000,79000000000,10003,1.2.643.5.1.13.13.12.2.1.13,nan,nan,17,16,...,1.0,450,057-657-352 98,17,10003,nan,1,1,1,0.0
3,2023,1,79000,79000000000,10003,1.2.643.5.1.13.13.12.2.1.13,nan,nan,17,16,...,1.0,450,068-037-801 71,17,10003,nan,1,1,1,0.0
4,2023,1,79000,79000000000,10003,1.2.643.5.1.13.13.12.2.1.13,nan,nan,17,16,...,1.0,450,130-244-079 03,17,10003,nan,1,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127665,2023,1,79000,79000000000,10022,1.2.643.5.1.13.13.12.2.1.27,nan,nan,49,68,...,2.1,100817,060-279-914 63,49,10022,nan,1,1,1,0.0
127666,2023,1,79000,79000000000,10022,1.2.643.5.1.13.13.12.2.1.27,nan,nan,90,112,...,1.0,468,155-283-978 96,90,10022,nan,1,1,1,0.0
127667,2023,1,79000,79000000000,10022,1.2.643.5.1.13.13.12.2.1.27,nan,nan,49,68,...,2.1,100817,060-279-914 63,49,10022,nan,1,1,1,0.0
127668,2023,1,79000,79000000000,10022,1.2.643.5.1.13.13.12.2.1.27,nan,nan,90,112,...,1.0,468,155-283-978 96,90,10022,nan,1,1,1,0.0


In [ ]:
# сохранение данных о количестве уникальных строк
s2 = df_data.drop_duplicates().shape[0]

## 2. Расчёт количества повторений каждой строки

In [18]:
# подчёт, сколько раз строка повторяется в наборе
# при этом набор данных "свёртывается", каждая срока повторяется только один раз, но в столбце size отображается количество повторов
df_data = df_data.groupby(df_data.columns.tolist(), as_index=False).size()
print(
    f"Количество уникальных строк в исходном наборе данных: {s2}, количество строк после 'свёртывания' набора: {df_data_size.shape[0]}"
)

Количество уникальных строк в исходном наборе данных: 26472, количество строк после 'свёртывания' набора: 26472


In [19]:
df_data

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size
0,2023,1,79000,79000000000,10001,1.2.643.5.1.13.13.12.2.1.32,nan,nan,3,4,...,447,149-003-926 52,3,10001,nan,1,1,1,0.0,3
1,2023,1,79000,79000000000,10001,1.2.643.5.1.13.13.12.2.1.32,nan,nan,3,4,...,447,149-003-926 52,3,10001,nan,1,1,1,0.0,5
2,2023,1,79000,79000000000,10001,1.2.643.5.1.13.13.12.2.1.32,nan,nan,3,4,...,447,149-003-926 52,3,10001,nan,1,1,1,0.0,7
3,2023,1,79000,79000000000,10001,1.2.643.5.1.13.13.12.2.1.32,nan,nan,3,4,...,447,149-003-926 52,3,10001,nan,1,1,1,0.0,5
4,2023,1,79000,79000000000,10001,1.2.643.5.1.13.13.12.2.1.32,nan,nan,3,4,...,401,149-003-926 52,3,10001,nan,1,1,2,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26467,2023,1,79000,79000000000,10046,1.2.643.5.1.13.13.12.2.1.12,nan,nan,104,158,...,494,203-691-760 56,104,10046,nan,1,1,2,0.0,1
26468,2023,1,79000,79000000000,10046,1.2.643.5.1.13.13.12.2.1.12,nan,nan,104,158,...,494,203-691-760 56,104,10046,nan,1,1,2,0.0,1
26469,2023,1,79000,79000000000,10046,1.2.643.5.1.13.13.12.2.1.12,nan,nan,104,158,...,494,203-691-760 56,104,10046,nan,1,1,2,0.0,1
26470,2023,1,79000,79000000000,10046,1.2.643.5.1.13.13.12.2.1.12,nan,nan,104,158,...,498,203-691-760 56,104,10046,nan,1,1,3,0.0,1


In [20]:
df_data.shape

(26472, 24)

## 3. Определение количества посещений

In [21]:
%%time
# определение количества посещений
df_data["visit_count"] = df_data.apply(visit_count, axis=1)

CPU times: user 463 ms, sys: 81 µs, total: 463 ms
Wall time: 461 ms


In [24]:
df_data

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count
0,2023,1,79000,79000000000,10001,1.2.643.5.1.13.13.12.2.1.32,nan,nan,3,4,...,149-003-926 52,3,10001,nan,1,1,1,0.0,3,3
1,2023,1,79000,79000000000,10001,1.2.643.5.1.13.13.12.2.1.32,nan,nan,3,4,...,149-003-926 52,3,10001,nan,1,1,1,0.0,5,5
2,2023,1,79000,79000000000,10001,1.2.643.5.1.13.13.12.2.1.32,nan,nan,3,4,...,149-003-926 52,3,10001,nan,1,1,1,0.0,7,7
3,2023,1,79000,79000000000,10001,1.2.643.5.1.13.13.12.2.1.32,nan,nan,3,4,...,149-003-926 52,3,10001,nan,1,1,1,0.0,5,5
4,2023,1,79000,79000000000,10001,1.2.643.5.1.13.13.12.2.1.32,nan,nan,3,4,...,149-003-926 52,3,10001,nan,1,1,2,0.0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26467,2023,1,79000,79000000000,10046,1.2.643.5.1.13.13.12.2.1.12,nan,nan,104,158,...,203-691-760 56,104,10046,nan,1,1,2,0.0,1,2
26468,2023,1,79000,79000000000,10046,1.2.643.5.1.13.13.12.2.1.12,nan,nan,104,158,...,203-691-760 56,104,10046,nan,1,1,2,0.0,1,2
26469,2023,1,79000,79000000000,10046,1.2.643.5.1.13.13.12.2.1.12,nan,nan,104,158,...,203-691-760 56,104,10046,nan,1,1,2,0.0,1,2
26470,2023,1,79000,79000000000,10046,1.2.643.5.1.13.13.12.2.1.12,nan,nan,104,158,...,203-691-760 56,104,10046,nan,1,1,3,0.0,1,3


In [28]:
# сохранение данных о количестве срок в исходном наборе
s1 = df_data.shape[0]

## 4. Расшифровка кодов

In [25]:
%%time

# загрузка справочников
f_032 = pd.read_excel("/home/sk27/work/med/data/F032.xlsx")
v_021 = pd.read_excel("/home/sk27/work/med/data/V021.xlsx")
v_002 = pd.read_excel("/home/sk27/work/med/data/V002.xlsx")
v_006 = pd.read_excel("/home/sk27/work/med/data/V006.xlsx")
v_025 = pd.read_excel("/home/sk27/work/med/data/V025.xlsx")
oid_data = pd.read_excel("/home/sk27/work/med/data/oid_data.xlsx")

# актуализация справочников
f_032 = f_032[["OKTMO_P", "SUBJ"]].drop_duplicates().reset_index(drop=True)
v_021 = v_021[v_021["DATEEND"].isna()]
v_002 = v_002[v_002["DATEEND"].isna()]
v_006 = v_006[v_006["DATEEND"].isna()]
v_025 = v_025[v_025["DATEEND"].isna()]

CPU times: user 46.2 s, sys: 273 ms, total: 46.5 s
Wall time: 46.5 s


In [26]:
# расшифорвка названия субьекта Росссийской Федерации
df_data = df_data.merge(f_032, left_on="oktmo", right_on="OKTMO_P", how="left")

In [29]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "depart",
        "branch",
        "doctor_specialty",
        "doctor_profile",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
        "visit_count",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [30]:
# расшифровка наименования (сокращённого) медучреждения
df_data = df_data.merge(oid_data, left_on="oid", right_on="oid", how="left")

In [31]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "doctor_profile",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
        "visit_count",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [32]:
# расшифровка специальности врача
df_data = df_data.merge(
    v_021, left_on="doctor_specialty", right_on="IDSPEC", how="left"
)

In [33]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME",
        "doctor_profile",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
        "visit_count",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [34]:
# расшифровка профиля врача
df_data = df_data.merge(v_002, left_on="doctor_profile", right_on="IDPR", how="left")

In [35]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME",
        "doctor_profile",
        "PRNAME",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
        "visit_count",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [36]:
# расшифровка специальности врача, оказавшего услугу
df_data = df_data.merge(
    v_021, left_on="doctor_specialty", right_on="IDSPEC", how="left"
)

In [37]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME_x",
        "doctor_profile",
        "PRNAME",
        "terms",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "POSTNAME_y",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
        "visit_count",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [38]:
# расшифровка условий оказания помощи
df_data = df_data.merge(v_006, left_on="terms", right_on="IDUMP", how="left")

In [39]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME_x",
        "doctor_profile",
        "PRNAME",
        "terms",
        "UMPNAME",
        "doctor_id",
        "diagnosis",
        "purpose",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "POSTNAME_y",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
        "visit_count",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [40]:
# расшифровка цели посещения
df_data = df_data.merge(v_025, left_on="purpose", right_on="IDPC", how="left")

In [41]:
df_data = df_data[
    [
        "year",
        "month",
        "region",
        "oktmo",
        "SUBJ",
        "clinic",
        "oid",
        "nameShort",
        "depart",
        "branch",
        "doctor_specialty",
        "POSTNAME_x",
        "doctor_profile",
        "PRNAME",
        "terms",
        "UMPNAME",
        "doctor_id",
        "diagnosis",
        "purpose",
        "N_PC",
        "service_id",
        "usl_service_id",
        "usl_doctor_specialty",
        "POSTNAME_y",
        "usl_clinic",
        "usl_depart",
        "sl_id",
        "code_usl",
        "kol_usl",
        "yet_sum",
        "size",
        "visit_count",
    ]
].reset_index(drop=True)
if df_data.shape[0] == s1:
    print("Число строк не изменилось")
else:
    print("Внимание! Изменилось число строк!")

Число строк не изменилось


In [42]:
df_data

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count
0,2023,1,79000,79000000000,Республика Адыгея,10001,1.2.643.5.1.13.13.12.2.1.32,ГБУЗ РА АРКБ,nan,nan,...,3,врач-аллерголог-иммунолог,10001,nan,1,1,1,0.0,3,3
1,2023,1,79000,79000000000,Республика Адыгея,10001,1.2.643.5.1.13.13.12.2.1.32,ГБУЗ РА АРКБ,nan,nan,...,3,врач-аллерголог-иммунолог,10001,nan,1,1,1,0.0,5,5
2,2023,1,79000,79000000000,Республика Адыгея,10001,1.2.643.5.1.13.13.12.2.1.32,ГБУЗ РА АРКБ,nan,nan,...,3,врач-аллерголог-иммунолог,10001,nan,1,1,1,0.0,7,7
3,2023,1,79000,79000000000,Республика Адыгея,10001,1.2.643.5.1.13.13.12.2.1.32,ГБУЗ РА АРКБ,nan,nan,...,3,врач-аллерголог-иммунолог,10001,nan,1,1,1,0.0,5,5
4,2023,1,79000,79000000000,Республика Адыгея,10001,1.2.643.5.1.13.13.12.2.1.32,ГБУЗ РА АРКБ,nan,nan,...,3,врач-аллерголог-иммунолог,10001,nan,1,1,2,0.0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26467,2023,1,79000,79000000000,Республика Адыгея,10046,1.2.643.5.1.13.13.12.2.1.12,"ГБУЗ РА ""АРПМР""",nan,nan,...,104,врач по медицинской реабилитации,10046,nan,1,1,2,0.0,1,2
26468,2023,1,79000,79000000000,Республика Адыгея,10046,1.2.643.5.1.13.13.12.2.1.12,"ГБУЗ РА ""АРПМР""",nan,nan,...,104,врач по медицинской реабилитации,10046,nan,1,1,2,0.0,1,2
26469,2023,1,79000,79000000000,Республика Адыгея,10046,1.2.643.5.1.13.13.12.2.1.12,"ГБУЗ РА ""АРПМР""",nan,nan,...,104,врач по медицинской реабилитации,10046,nan,1,1,2,0.0,1,2
26470,2023,1,79000,79000000000,Республика Адыгея,10046,1.2.643.5.1.13.13.12.2.1.12,"ГБУЗ РА ""АРПМР""",nan,nan,...,104,врач по медицинской реабилитации,10046,nan,1,1,3,0.0,1,3


## 4. Контроль результатов

### Не расшифрованные данные:

In [43]:
# не расшифрованы данные о субъекте
df_data.loc[df_data[df_data["SUBJ"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [44]:
# не расшифрованы данные о наименовании медучреждения
df_data.loc[df_data[df_data["nameShort"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [45]:
# не расшифрованы данные о специальности врача
df_data.loc[df_data[df_data["POSTNAME_x"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [46]:
# не расшифрованы данные о профиле врача
df_data.loc[df_data[df_data["PRNAME"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [47]:
# не расшифрованы данные о специальности врача, оказавшего услугу
df_data.loc[df_data[df_data["POSTNAME_y"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [48]:
# не расшифрованы данные об условиях оказания помощи
df_data.loc[df_data[df_data["UMPNAME"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [49]:
# не расшифрованы данные оцели посещения
df_data.loc[df_data[df_data["N_PC"].isna()].index]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [50]:
# формат данных
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26472 entries, 0 to 26471
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  26472 non-null  int64  
 1   month                 26472 non-null  int64  
 2   region                26472 non-null  int64  
 3   oktmo                 26472 non-null  int64  
 4   SUBJ                  26472 non-null  object 
 5   clinic                26472 non-null  int64  
 6   oid                   26472 non-null  object 
 7   nameShort             26472 non-null  object 
 8   depart                26472 non-null  object 
 9   branch                26472 non-null  object 
 10  doctor_specialty      26472 non-null  int64  
 11  POSTNAME_x            26472 non-null  object 
 12  doctor_profile        26472 non-null  int64  
 13  PRNAME                26472 non-null  object 
 14  terms                 26472 non-null  int64  
 15  UMPNAME            

In [51]:
# пустые количества посещений
df_data[np.isnan(df_data["visit_count"])]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [52]:
# не определённые количества посещений
df_data[df_data["visit_count"] == "не определено"]

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count


In [53]:
# сохранение результатов на диск
df_data.to_csv("/home/sk27/work/med/data/result.csv", encoding="utf-8-sig")

## "Свёртывание" файла

In [112]:
df_data = pd.read_csv(
    "/home/sk27/work/med/new/results/03_04_2023/ryazan.csv",
    sep=",",
    # encoding="ISO-8859-1",
    index_col=0,
    low_memory=False,
)

In [113]:
df_data

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count
0,2022,9,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,35,врач-невролог,620077,073_2,1,1,1,0.0,1,1
1,2022,10,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,35,врач-невролог,620077,073_2,1,1,1,0.0,1,1
2,2022,10,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,35,врач-невролог,620077,073_2,1,1,1,0.0,1,1
3,2022,10,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,35,врач-невролог,620077,073_2,1,1,1,0.0,1,1
4,2022,10,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,35,врач-невролог,620077,073_2,1,1,1,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51289,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,81,врач ультразвуковой диагностики,620105,291_2,26,26,1,0.0,1,26
51290,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,93,врач-эндоскопист,620105,291_2,9,9,1,0.0,1,9
51291,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,93,врач-эндоскопист,620105,291_2,11,11,1,0.0,1,11
51292,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,93,врач-эндоскопист,620105,291_2,1,1,1,0.0,1,1


In [114]:
df_data["visit_count"].sum()

406557

In [115]:
df_data = df_data.drop_duplicates()

In [116]:
df_data.shape

(51294, 32)

In [117]:
df_data["full_visit_count"] = df_data["size"] * df_data["visit_count"]

In [118]:
df_data["full_visit_count"].sum()

406557

In [119]:
df_data = df_data.drop(columns="visit_count").reset_index(drop=True)
df_data = df_data.rename(columns={"full_visit_count": "visit_count"})

In [120]:
df_data

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,usl_doctor_specialty,POSTNAME_y,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,size,visit_count
0,2022,9,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,35,врач-невролог,620077,073_2,1,1,1,0.0,1,1
1,2022,10,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,35,врач-невролог,620077,073_2,1,1,1,0.0,1,1
2,2022,10,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,35,врач-невролог,620077,073_2,1,1,1,0.0,1,1
3,2022,10,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,35,врач-невролог,620077,073_2,1,1,1,0.0,1,1
4,2022,10,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,35,врач-невролог,620077,073_2,1,1,1,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51289,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,81,врач ультразвуковой диагностики,620105,291_2,26,26,1,0.0,1,26
51290,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,93,врач-эндоскопист,620105,291_2,9,9,1,0.0,1,9
51291,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,93,врач-эндоскопист,620105,291_2,11,11,1,0.0,1,11
51292,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,93,врач-эндоскопист,620105,291_2,1,1,1,0.0,1,1


In [121]:
df_data.shape

(51294, 32)

In [122]:
# сохранение результатов на диск
df_data.to_csv(
    "/home/sk27/work/med/new/results/03_04_2023/short_results/ryazan.csv",
    encoding="utf-8-sig",
)